In [339]:
# Setup + Helper Methods

import pandas as pd
import snap

# return list of graphs
def get_graph_database():
    G1 = snap.LoadEdgeList(snap.TUNGraph, "datasets/test-graphs/graph-A.txt", 0, 1)
    G2 = snap.LoadEdgeList(snap.TUNGraph, "datasets/test-graphs/graph-B.txt", 0, 1)
    G3 = snap.LoadEdgeList(snap.TUNGraph, "datasets/test-graphs/graph-C.txt", 0, 1)
    G4 = snap.LoadEdgeList(snap.TUNGraph, "datasets/test-graphs/graph-D.txt", 0, 1)
    graph_database = [G1, G2, G3, G4]   
    
    return graph_database


'''
NOTE: (get_all_node_supports) 
This method assumes that there are no label repitions in any of the graphs.
That is, none of the graphs have more than one node with ID x.
'''
# return dict with all nodes in g and their support
def get_all_node_supports(graph_database):
    NS = {}
    
    for graph in graph_database:
        for N in graph.Nodes():
            curr_node = N.GetId()
            if curr_node in NS:
                NS[curr_node] += 1
            else:
                NS[curr_node] = 1
    
    return NS

    
'''
NOTE: (get_all_edge_supports)
This methods takes duplication into account.
That is, the edges NodeX-NodeY and NodeY-NodeX are considered the same.
''' 
# return dict will all edges in g and their supports
def get_all_edge_supports(graph_database):
        ES = {}    
        
        for graph in graph_database:
            for E in graph.Edges():
                curr_edge = (E.GetSrcNId(), E.GetDstNId())
                curr_edge_flip = (E.GetDstNId(), E.GetSrcNId)
                if curr_edge in ES:
                    ES[curr_edge] += 1
                elif curr_edge_flip in ES:
                    ES[curr_edge_flip] += 1
                else:    
                    ES[curr_edge] = 1
        
        return ES
    
    
# return all graph edges as list
def graph_to_list(graph):
    graph_list = []
    
    if graph.GetEdges() == 0:
        for N in graph.Nodes():
            graph_list.append(N.GetId())
    else:
        for E in graph.Edges():
            curr_edge = (E.GetSrcNId(), E.GetDstNId())
            graph_list.append(curr_edge)
    
    return graph_list
         
    
# print graphs and their supports / print dict
def print_dict(D, opt):
    if opt == "graph":
        for graph in D.keys():
            print("Graph: {}, Support: {}".format(graph_to_list(graph), D[graph]))
        
    else:
        for key in D.keys():
            print("Key : {} , Value : {}".format(key, D[key]))
            

In [330]:
# Node-based Join Growth

'''
To Do:
def check_downward_closure(candidate): 
''' 

# return dict with frequent singleton graphs and their supports
def get_frequent_singleton_graphs(NS, minsup):
    F1 = {}
    
    for N in NS:
        if NS[N] >= minsup:
            subgraph = snap.TUNGraph.New() # create new graph
            subgraph.AddNode(N) # add frequent node
            F1[subgraph] = NS[N] # graph support = node support
    
    return F1


# return candidate by joining singletons
def join_singletons(subgraph1, subgraph2):
    c = snap.TUNGraph.New() # new candidate subgraph
    c = snap.ConvertGraph(type(subgraph1), subgraph1)
    
    c.AddNode(subgraph2.BegNI().GetId()) # add subgraph2 node to subgraph1
    c.AddEdge(subgraph1.BegNI().GetId(), subgraph2.BegNI().GetId()) # add edge between nodes
        
    return c


'''
NOTE: (subgraph_match)
The book recommends Ullman's Algorithm for this,
which is recursive. I hate recursion so I made the following,
which might not be as efficient but does the job.
'''
# return node and edge if they are the only non-matching ones
def subgraph_match(Gq, G):
    nmE_f = False # non-matching node found
    nmN_f = False # non-matching edge found
    nmN = None # non_matching node
    nmE = None # non-matching edge

    res = False

    for E in Gq.Edges():
        if not G.IsEdge(E.GetSrcNId(), E.GetDstNId()):
            if nmE_f:
                nmE_f = False
                break
            else:
                nmE = (E.GetSrcNId(), E.GetDstNId())
                nmE_f = True
    
    if nmE_f:
        for N in Gq.Nodes():
            if not G.IsNode(N.GetId()):
                if nmN_f:
                    nmN_f = False
                    break
                else:
                    nmN = N.GetId()
                    if nmN in nmE:
                        nmN_f = True
                    else:
                        break
                        
    if nmN_f and nmE_f:
        res = True
    
    return res, nmE, nmN


# return candidates by performing node-based joins    
def join_subgraphs(subgraph1, subgraph2, nmE, nmN):
    # create new candidate subgraphs
    c1 = snap.TUNGraph.New()
    c2 = snap.TUNGraph.New()
    
    # hold non-matching node in subgraph2
    for N in subgraph2.Nodes():
        if not subgraph1.IsNode(N.GetId()):
            nmN_s2 = N.GetId()
    
    c1 = snap.ConvertGraph(type(subgraph2), subgraph2) # copy subgraph1
    c1.AddNode(nmN) # add non-matching node from subgraph1
    c1.AddEdge(nmE[0], nmE[1]) # add non-matching edge
    
    c2 = snap.ConvertGraph(type(c1), c1) # copy candidate 1
    c2.AddEdge(nmN, nmN_s2) # add edge between non-matching nodes of subgraph1 and subgraph2
    
    #--------------------------- Debug:
#     print("Joined Candidate 1:")
#     print_graph(c1)
#     print("Joined Candidate 2:")
#     print_graph(c2)
    #---------------------------
    
    return c1, c2


'''
NOTE: (generate_candidates)
Method is incomplete i.e. no optimization/pruning of candidates'
Which is why duplicate candidates are created and final output
contains duplicate graphs.
'''
def generate_candidates(Fk, k):
    candidates = []
    
    for i in range(0, len(Fk)):
        sG1 = Fk[i]
        
        for j in range(i+1, len(Fk)):
            sG2 = Fk[j]

            if k == 2:
                c = join_singletons(sG1, sG2)
                candidates.append(c)

            else:
                match, nmE, nmN = subgraph_match(sG1, sG2)
                #--------------------------- Debug:
#                 print(match)
#                 print("sG1: ")
#                 print_graph(sG1)
#                 print("sG2: ")
#                 print_graph(sG2)
                #---------------------------
                if match:
                    c1, c2 = join_subgraphs(sG1, sG2, nmE, nmN)
                    for _c in c1, c2:
                        candidates.append(_c)
                
    return candidates

# generate frequent k+1 sized graphs dict by counting C in g
def generate_Fkplus1(C, g, minsup):
    Fkplus1 = {}
    candidate_is_subgraph = True
    support = 0
    
    for candidate in C:
        
        for graph in g:
            
            # check if graph contains all candidate's nodes
            for N in candidate.Nodes():
                if not graph.IsNode(N.GetId()):
                    candidate_is_subgraph = False
                    break
                    
            # check if graph contains all candidate's edges    
            if candidate_is_subgraph:        
                for E in candidate.Edges():
                    if not graph.IsEdge(E.GetSrcNId(), E.GetDstNId()):
                        candidate_is_subgraph = False
                        break
                        
            # increment support            
            if candidate_is_subgraph:
                support += 1
            
            # reset for next graph
            candidate_is_subgraph = True
    
        if support >= minsup:
            Fkplus1[candidate] = support
        
        support = 0 # reset support for next candidate
        
    #--------------------------- Debug:    
#     print("Fkplus1: ")
#     print_dict(Fkplus1, "graph")
    #---------------------------
    
    return Fkplus1



# return dict with frequent subgraphs and support
def node_based_join_growth(g, minsup):
    
    # NS = { [(NodeId) : Support] }
    NS = get_all_node_supports(g)
    
    # F1 = { All frequent singleton graphs }
    Fk = get_frequent_singleton_graphs(NS, minsup) # frequent k subgraphs
    k = 1
    
    FsG = {} # all frequent subgraphs
    C = [] # candidates from Fk
    
    # Apriori Algorithm:
    while(True):
        C = generate_candidates(list(Fk.keys()), k+1)
        Fk = generate_Fkplus1(C, g, minsup)
        
        # end if no more frequent subgraphs
        if not Fk: 
            break
            
        FsG.update(Fk) # append all frequent subgraphs
        
        k = k + 1
    
    return FsG

In [ ]:
# Edge-based Join Growth

In [337]:
# Main

# minsup = minimum support
minsup = 2

# g = [G1, G2, G3 ... G10] 
g = get_graph_database()

# Skaramoosh
FsG = node_based_join_growth(g, minsup)

print("FsG: ")
print_dict(FsG, "graph")

FsG: 
Graph: [(1, 5)], Support: 4
Graph: [(1, 12)], Support: 4
Graph: [(5, 6)], Support: 4
Graph: [(4, 12)], Support: 4
Graph: [(13, 19)], Support: 4
Graph: [(19, 20)], Support: 4
Graph: [(3, 8)], Support: 4
Graph: [(3, 9)], Support: 4
Graph: [(3, 11)], Support: 4
Graph: [(8, 9)], Support: 4
Graph: [(4, 6)], Support: 4
Graph: [(13, 18)], Support: 4
Graph: [(18, 20)], Support: 4
Graph: [(89, 100)], Support: 2
Graph: [(1, 5), (1, 12)], Support: 4
Graph: [(5, 6), (1, 5)], Support: 4
Graph: [(4, 12), (1, 12)], Support: 4
Graph: [(4, 6), (5, 6)], Support: 4
Graph: [(4, 6), (4, 12)], Support: 4
Graph: [(19, 20), (13, 19)], Support: 4
Graph: [(13, 18), (13, 19)], Support: 4
Graph: [(18, 20), (19, 20)], Support: 4
Graph: [(3, 8), (3, 9)], Support: 4
Graph: [(3, 8), (3, 9), (8, 9)], Support: 4
Graph: [(3, 8), (3, 11)], Support: 4
Graph: [(8, 9), (3, 8)], Support: 4
Graph: [(8, 9), (3, 8), (3, 9)], Support: 4
Graph: [(3, 9), (3, 11)], Support: 4
Graph: [(8, 9), (3, 9)], Support: 4
Graph: [(8, 9)

In [274]:
# Trials and Testing

subgraph1 = snap.TUNGraph.New()
subgraph1.AddNode(1)
subgraph1.AddNode(6)
subgraph1.AddNode(7)
subgraph1.AddEdge(1, 6)
subgraph1.AddEdge(1, 7)

subgraph2 = snap.TUNGraph.New()
subgraph2.AddNode(1)
subgraph2.AddNode(12)
subgraph2.AddNode(5)
subgraph2.AddEdge(1, 12)
subgraph2.AddEdge(1, 5)

res = subgraph_match(subgraph1, subgraph2)

print(res)

#--------------------------- Debug:
# for G in FsG.keys():
#     if G.GetEdges() > 3:
#         print_graph(G)
#         print("Support: {}".format(FsG[G]))
#---------------------------

(False, (1, 6))
